* Master DAC - BDLE
* Author: Mohamed-Amine Baazizi
* Affiliation: LIP6 - Faculté des Sciences - Sorbonne Université
* Email: mohamed-amine.baazizi@lip6.fr
* October 2024


FERCHE Adelin-Flaviu \
3800655 \
M2 SAR

# Map-Reduce in Spark

The goal of this lab is to write programs in the Map-Reduce paradigm using the RDD-python API of Spark.
The API for documentation is available here:

- https://spark.apache.org/docs/latest/api/python/reference/pyspark.html


You are expected to use the following higher order functions like `map, flatMap, filter, reduceByKey, groupByKey, join, zipWithIndex` and the standard relational operators `union, intersection, subtract, join` and to favor functional-style programming.

## Pre-requisite

#### Installation

In [1]:
%%capture
!pip install -q pyspark

In [2]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

local = "local[*]"
appName = "Spark-RDD"
localConfig = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G")

spark = SparkSession.builder.config(conf = localConfig).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

### Data loading
Create a sub-directory `ens/data/SparkMR` into your storage space in Google Drive than download this [file](https://drive.google.com/file/d/1sqCeOKJ38E2Nc67Kn19ilKlHsBvTQAtW/view?usp=sharing) into this sub-directory.

In [3]:
! mkdir -p /tmp/spark-rdd/

In [4]:
! wget https://nuage.lip6.fr/s/BMHNNgi9YoKCmCi/download/wordcount.txt -O /tmp/spark-rdd/wordcount.txt

--2024-10-28 15:59:11--  https://nuage.lip6.fr/s/BMHNNgi9YoKCmCi/download/wordcount.txt
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 799738 (781K) [text/plain]
Saving to: ‘/tmp/spark-rdd/wordcount.txt’

/tmp/spark-rdd/word 100%[===================>] 780.99K  1.02MB/s    in 0.8s    

2024-10-28 15:59:13 (1.02 MB/s) - ‘/tmp/spark-rdd/wordcount.txt’ saved [799738/799738]



In [5]:
! ls /tmp/spark-rdd/wordcount.txt

/tmp/spark-rdd/wordcount.txt


In [6]:
#display n elements
n=10

In [7]:
wc_path = '/tmp/spark-rdd/wordcount.txt'

## Simplified TF*IDF

Write instructions to compute the simplified version of TF*IDF for a synthetic corpus of words.
The input data is an RDD of pairs $$(d_i, [w_1,\ldots,w_{n_i}])$$
where:
* $d_i$ is a document identifier and
* $[w_1,\ldots,w_{n_i}]$ a list of words associated to $d_i$  with possible repetition.

You are expected to produce an RDD of triples $$(d_l, w_m,  tf \times idf)$$
where:
* $tf$ is the frequency of $w_m$ in $d_l$ and
* $idf$ is the number of documents where $w_m$ appears

In [8]:
dataset = ['d1,one fish two fish','d2,red fish blue', 'd3,one red bird']
data = spark.sparkContext.parallelize(dataset)
lines = data.filter(lambda x: x!='')\
.map(lambda x : x.split(",")).map(lambda x:(x[0],x[1].split(" ")))
lines.take(n)

[('d1', ['one', 'fish', 'two', 'fish']),
 ('d2', ['red', 'fish', 'blue']),
 ('d3', ['one', 'red', 'bird'])]

In [9]:
count = data.count()
count

3

complete the stub of `combine` which is a python function that combines each word in `list` with `word` given as argument

In [10]:
def combine(word, list):
  res = []
  for w in list:
    res.append((word,w))
  return res

In [11]:
list = ['one', 'red', 'bird']
word = 'd1'
combine(word,list)

[('d1', 'one'), ('d1', 'red'), ('d1', 'bird')]

starting from `lines` produce and RDD of all pairs $$(d_i, w_j)$$

In [12]:
word_doc = lines.flatMap(lambda x: combine(x[0], x[1]))
word_doc.take(n)

[('d1', 'one'),
 ('d1', 'fish'),
 ('d1', 'two'),
 ('d1', 'fish'),
 ('d2', 'red'),
 ('d2', 'fish'),
 ('d2', 'blue'),
 ('d3', 'one'),
 ('d3', 'red'),
 ('d3', 'bird')]

In [13]:
word_doc = lines.flatMap(lambda x: combine(x[0], x[1]))
word_doc.take(n)

[('d1', 'one'),
 ('d1', 'fish'),
 ('d1', 'two'),
 ('d1', 'fish'),
 ('d2', 'red'),
 ('d2', 'fish'),
 ('d2', 'blue'),
 ('d3', 'one'),
 ('d3', 'red'),
 ('d3', 'bird')]

In [14]:
def add(a,b):
  return a+b

For each pair (document,word), count its frequency

In [15]:
freq_word_doc = word_doc.map(lambda x: ((x[0], x[1]), 1)).reduceByKey(add)
freq_word_doc.take(n)

[(('d1', 'one'), 1),
 (('d2', 'fish'), 1),
 (('d3', 'one'), 1),
 (('d3', 'red'), 1),
 (('d1', 'fish'), 2),
 (('d1', 'two'), 1),
 (('d2', 'red'), 1),
 (('d2', 'blue'), 1),
 (('d3', 'bird'), 1)]

for each word, produce a pair (d,freq)

In [16]:
freq_word = freq_word_doc.map(lambda x: (x[0][1], (x[0][0], x[1])))
freq_word.take(n)

[('one', ('d1', 1)),
 ('fish', ('d2', 1)),
 ('one', ('d3', 1)),
 ('red', ('d3', 1)),
 ('fish', ('d1', 2)),
 ('two', ('d1', 1)),
 ('red', ('d2', 1)),
 ('blue', ('d2', 1)),
 ('bird', ('d3', 1))]

for each word, compute its document frequency, i.e the number of documents in which it appears

In [17]:
freq_doc = freq_word.map(lambda x: (x[0], x[1][1])).reduceByKey(add).map(lambda x: (x[0], 3 / x[1]))
freq_doc.take(n)

[('fish', 1.0),
 ('two', 3.0),
 ('bird', 3.0),
 ('one', 1.5),
 ('red', 1.5),
 ('blue', 3.0)]

for each word, compute its simplified score

In [18]:
final = freq_word.join(freq_doc).map(lambda x: (x[0], x[1][0][0], x[1][1]))
final.take(n)

[('two', 'd1', 3.0),
 ('bird', 'd3', 3.0),
 ('red', 'd3', 1.5),
 ('red', 'd2', 1.5),
 ('blue', 'd2', 3.0),
 ('fish', 'd2', 1.0),
 ('fish', 'd1', 1.0),
 ('one', 'd1', 1.5),
 ('one', 'd3', 1.5)]

## Text analytics

The goal is to compute the SPMI (simplified pointwise mutual information) metric for words in a corpus.
Given two words, a and b, we call $ab$ a *bigram* when $a$ preceds $b$ in the corpus.
We would like to implement the following formulae:  

$$SPMI(a, b) =  \frac{P(ab)}{P(a) * P(b)}$$
where:

* $a$ and $b$ are words from the corpus,
* $P(ab)$ is the frequency of the bigram $ab$ relatively to the number of all *distinct* bigrams in the coprus and
* $P(a)$, resp. $P(b)$ is the frequency of the word $a$, resp. $b$ relatively to all *distinct* words from the corpus


In [19]:
def removespecialchars (word) :
    charstoremove = ['[',']','*','#','.','_',':','?','!',',',';','“','”','\n']
    for ch in word:
        if ch in charstoremove:
            word = word.replace(ch,'')
    return word

Read file in utf-8 using use_unicode=False

In [20]:
data = spark.sparkContext.textFile(wc_path, use_unicode="False")
print("before filtering there are %d lines" % data.count())
# data.take(n)
lines = data.filter(lambda x: x!='')\
.map(lambda x:removespecialchars(x))\
.map(lambda x : x.split(" "))
print("after filtering there are %d lines" % lines.count())


before filtering there are 14594 lines
after filtering there are 12153 lines


retrieve the list of 10 first words

In [21]:
words = lines.flatMap(lambda x: x)
words.take(n)

['The',
 'Project',
 'Gutenberg',
 'EBook',
 'of',
 'Pride',
 'and',
 'Prejudice',
 'by',
 'Jane']

compute the frequency of each word

In [22]:
wordcount = words.map(lambda x: (x, 1)).reduceByKey(add)
wordcount.take(n)

[('The', 285),
 ('Project', 83),
 ('EBook', 3),
 ('of', 3693),
 ('Pride', 6),
 ('Jane', 263),
 ('Austen', 4),
 ('is', 859),
 ('use', 23),
 ('anyone', 29)]

count the number of distinct words

In [23]:
nb_distinct_words = wordcount.count()
nb_distinct_words

7762

compute the proba of each word relatively to the number of distinct words

In [24]:
word_proba = wordcount.map(lambda x: (x[0], x[1] / nb_distinct_words))
word_proba.take(n)

[('The', 0.0367173408915228),
 ('Project', 0.010693120329811904),
 ('EBook', 0.00038649832517392423),
 ('of', 0.47577943828910074),
 ('Pride', 0.0007729966503478485),
 ('Jane', 0.03388301984024736),
 ('Austen', 0.000515331100231899),
 ('is', 0.1106673537748003),
 ('use', 0.0029631538263334193),
 ('anyone', 0.0037361504766812675)]

Collocation of words: build an RDD of bigrams. A bigram is a pair of words appearing in consecutive order.

In [37]:
words_index = words.filter(lambda x: x != '').zipWithIndex().map(lambda x: (x[1], x[0]))
words_index.take(n)

[(0, 'The'),
 (1, 'Project'),
 (2, 'Gutenberg'),
 (3, 'EBook'),
 (4, 'of'),
 (5, 'Pride'),
 (6, 'and'),
 (7, 'Prejudice'),
 (8, 'by'),
 (9, 'Jane')]

In [38]:
words_index_other = words_index.map(lambda x: (x[0] - 1, x[1]))
words_index_other.take(n)

[(-1, 'The'),
 (0, 'Project'),
 (1, 'Gutenberg'),
 (2, 'EBook'),
 (3, 'of'),
 (4, 'Pride'),
 (5, 'and'),
 (6, 'Prejudice'),
 (7, 'by'),
 (8, 'Jane')]

In [39]:
bigram = words_index.join(words_index_other).map(lambda x: x[1])
bigram.take(n)

[('The', 'Project'),
 ('of', 'Pride'),
 ('by', 'Jane'),
 ('eBook', 'is'),
 ('use', 'of'),
 ('at', 'no'),
 ('with', 'almost'),
 ('whatsoever', 'You'),
 ('it', 'give'),
 ('or', 're-use')]

count the fequency of each bigram

In [40]:
bigramcount = bigram.map(lambda x: (x, 1)).reduceByKey(add)
bigramcount.take(n)

[(('use', 'of'), 8),
 (('at', 'no'), 6),
 (('the', 'terms'), 14),
 (('12', '2019'), 1),
 (('PREJUDICE', 'Produced'), 1),
 (('Volunteers', 'and'), 3),
 (('and', 'Prejudice'), 4),
 (('Chapter', '4'), 2),
 (('Chapter', '10'), 2),
 (('Chapter', '12'), 2)]

attach to each bigram the probability of its words, i.e. $$P(a) * P(b)$$

In [55]:
bigramproba = bigramcount.map(lambda x: x[0]) \
                         .join(word_proba) \
                         .map(lambda x: (x[1][0], (x[0], x[1][1]))) \
                         .join(word_proba) \
                         .map(lambda x: ((x[1][0][0], x[0]), x[1][0][1] * x[1][1]))
bigramproba.take(n)

[(('of', 'Pride'), 0.0003677759121018558),
 (('Title', 'Pride'), 9.958730357483232e-08),
 (('vain', 'Pride'), 1.7925714643469818e-06),
 (('mine', 'Pride'), 1.4938095536224848e-06),
 (('cover', 'Pride'), 1.9917460714966465e-07),
 (('of', 'them'), 0.026602457642034234),
 (('like', 'them'), 0.0005474646035187116),
 (('seen', 'them'), 0.0005402611218934654),
 (('before', 'them'), 0.00161357988405515),
 (('make', 'them'), 0.0012101849130413626)]

count the number of distinct bigrams

In [45]:
bigram_distinct = bigramcount.count()
bigram_distinct

59099

attach to each bigram its probability, i.e. $$P(ab)$$

In [46]:
bigramjointproba = bigramcount.map(lambda x: (x[0], x[1] / bigram_distinct))
bigramjointproba.take(n)

[(('use', 'of'), 0.00013536608064434254),
 (('at', 'no'), 0.0001015245604832569),
 (('the', 'terms'), 0.00023689064112759946),
 (('12', '2019'), 1.6920760080542818e-05),
 (('PREJUDICE', 'Produced'), 1.6920760080542818e-05),
 (('Volunteers', 'and'), 5.076228024162845e-05),
 (('and', 'Prejudice'), 6.768304032217127e-05),
 (('Chapter', '4'), 3.3841520161085635e-05),
 (('Chapter', '10'), 3.3841520161085635e-05),
 (('Chapter', '12'), 3.3841520161085635e-05)]

compute the SPMI and sort in descending order on the score

In [58]:
SPMI = bigramjointproba.join(bigramproba).map(lambda x: (x[0], x[1][0] / x[1][1])).sortBy(lambda x: x[1], False)
SPMI.take(n)

[(('staff', 'Please'), 1019.4528503020354),
 (('he—poor', 'Eliza—to'), 1019.4528503020354),
 (('net', 'purses'), 1019.4528503020354),
 (('Italian', 'songs'), 1019.4528503020354),
 (('barbarously', 'misused'), 1019.4528503020354),
 (('weighty', 'accusation'), 1019.4528503020354),
 (('vicious', 'propensities—the'), 1019.4528503020354),
 (('untamed', 'unabashed'), 1019.4528503020354),
 (('City', 'UT'), 1019.4528503020354),
 (('Kenilworth', 'Birmingham'), 1019.4528503020354)]

## Matrix multiplication

Write a program to perform matrix multiplication using the Spark RDD operators.
The input matrices are represented with their coordinates: each matrix is an RDD of tuples `(i,j,val)` where `i` is the line number, `j` the column number and `val` the value.
The input matrices are provided (`matM` and `matN`).
The output matrix is also represented with its coordinates:

In [60]:
M = [(1,1,1),(1,2,2),(1,3,3),(2,1,2),(2,2,5),(2,3,7)]
N = [(1,1,2),(1,2,4),(1,3,8),(2,1,1),(2,2,5),(2,3,10),(3,1,3),(3,2,6),(3,3,9)]

matM = spark.sparkContext.parallelize(M)
matN = spark.sparkContext.parallelize(N)

In [67]:
matMElem = matM.map(lambda x: (x[1], (x[0], x[2])))
matNElem = matN.map(lambda x: (x[0], (x[1], x[2])))
pairwiseProd = matMElem.join(matNElem).map(lambda x: ((x[1][0][0], x[1][1][0]), x[1][0][1] * x[1][1][1]))
product = pairwiseProd.reduceByKey(lambda a,b:a+b)
product.collect()

[((1, 1), 13),
 ((1, 2), 32),
 ((2, 3), 129),
 ((1, 3), 55),
 ((2, 2), 75),
 ((2, 1), 30)]

##END